##Install required tools

In [1]:
!pip install git+https://github.com/openai/whisper.git 
!pip install onnx
!pip install onnx_tf
!git clone https://github.com/openai/whisper.git
!git clone https://github.com/usefulsensors/openai-whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-9qlfj4nk
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-9qlfj4nk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'whisper' already exists and is not an empty directory.
fatal: destination path 'openai-whisper' already exists and is not an empty directory.


##Generate whisper encoder tflite(int8) model and run Inference 

In [2]:
import whisper
import torch
import tensorflow as tf
import onnx
import numpy as np
import argparse
import os
import warnings
import tqdm
from onnx_tf.backend import prepare
from whisper.audio import load_audio, log_mel_spectrogram,pad_or_trim,N_FRAMES, SAMPLE_RATE

#load openai->whisper(pytorch)->tiny model
tiny_model = whisper.load_model("tiny")

#Export to onnx format
torch.onnx.export(tiny_model.encoder, torch.randn(1,80,3000).to("cpu"), "./whisper-encoder.onnx")
#torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258]]).to("cuda"), torch.randn(1,384,384).to("cuda")), "./whisper-decoder_main.onnx")
#torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258, 50259, 50359]]).to("cuda"), torch.randn(1, 384, 384).to("cuda")), "./whisper-decoder_language.onnx")
onnx_model_path = './whisper-encoder.onnx'
tf_model_path = 'model_tf-encoder'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)


saved_model_dir = 'model_tf-encoder'
tflite_model_path = 'whisper-encoder-int8.tflite'

def representative_dataset_random():
    for _ in range(100):
      data = np.random.rand(1, 80, 3000)
      yield [data.astype(np.float32)]

def representative_dataset():
    for _ in range(10):#Change this to 100 and provide 100 different audio files 
      mel_from_file = log_mel_spectrogram('/content/whisper/tests/jfk.flac')
      segment = pad_or_trim(mel_from_file, N_FRAMES)
      segment = tf.expand_dims(segment, 0)
      print(segment.shape)
      yield [segment]

# Convert to tflite(int8) model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.representative_dataset = representative_dataset
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
import tensorflow as tf
import numpy as np
tflite_model_path = './whisper-encoder-int8.tflite'

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

print("== Input details ==")
print("name:", interpreter.get_input_details()[0]['name'])
print("shape:", interpreter.get_input_details()[0]['shape'])
print("type:", interpreter.get_input_details()[0]['dtype'])

print("\nDUMP INPUT")
print(interpreter.get_input_details()[0])

print("\n== Output details ==")
print("name:", interpreter.get_output_details()[0]['name'])
print("shape:", interpreter.get_output_details()[0]['shape'])
print("type:", interpreter.get_output_details()[0]['dtype'])

print("\nDUMP OUTPUT")
print(interpreter.get_output_details()[0])

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model with random data
input_shape = input_details[0]['shape']
mel_from_file = log_mel_spectrogram('/content/whisper/tests/jfk.flac')
input_tensor = pad_or_trim(mel_from_file, N_FRAMES)
input_tensor = tf.expand_dims(input_tensor, 0)
input_tensor = np.array(input_tensor-128, dtype=np.int8)
interpreter.set_tensor(input_details[0]['index'], input_tensor)

interpreter.invoke()
print("Inference executed successfully\n")

/usr/local/lib/python3.7/dist-packages/whisper/model.py:152: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert x.shape[1:] == self.positional_embedding.shape, "incorrect audio shape"
/usr/local/lib/python3.7/dist-packages/whisper/model.py:90: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  scale = (n_state // self.n_head) ** -0.25


(1, 80, 3000)
(1, 80, 3000)
(1, 80, 3000)
(1, 80, 3000)
(1, 80, 3000)
(1, 80, 3000)
(1, 80, 3000)
(1, 80, 3000)
(1, 80, 3000)
(1, 80, 3000)
== Input details ==
name: serving_default_x.1:0
shape: [   1   80 3000]
type: <class 'numpy.int8'>

DUMP INPUT
{'name': 'serving_default_x.1:0', 'index': 0, 'shape': array([   1,   80, 3000], dtype=int32), 'shape_signature': array([   1,   80, 3000], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.007843137718737125, -59), 'quantization_parameters': {'scales': array([0.00784314], dtype=float32), 'zero_points': array([-59], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}

== Output details ==
name: PartitionedCall:0
shape: [   1 1500  384]
type: <class 'numpy.int8'>

DUMP OUTPUT
{'name': 'PartitionedCall:0', 'index': 569, 'shape': array([   1, 1500,  384], dtype=int32), 'shape_signature': array([   1, 1500,  384], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.1381371021270752, -2), 'quantization_

## Convert from pytorch to onnx

In [3]:
import whisper
import torch

tiny_model = whisper.load_model("tiny")
torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258]]).to("cpu"), torch.randn(1,384,384).to("cpu")), "/content/whisper-decoder_main.onnx")
torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258, 50259, 50359]]).to("cpu"), torch.randn(1, 384, 384).to("cpu")), "/content/whisper-decoder_language.onnx")

/usr/local/lib/python3.7/dist-packages/whisper/model.py:90: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  scale = (n_state // self.n_head) ** -0.25


##Decoder TFLite model

In [4]:
from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-decoder_main.onnx'
tf_model_path = 'model_tf-decoder_main'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)
saved_model_dir = 'model_tf-decoder_main'
tflite_model_path = 'whisper-decoder_main.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

##Decoder language TFLite model

In [5]:
from onnx_tf.backend import prepare
import onnx

onnx_model_path = '/content/whisper-decoder_language.onnx'
tf_model_path = 'model_tf-decoder_language'

onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)
saved_model_dir = 'model_tf-decoder_language'
tflite_model_path = 'whisper-decoder_language.tflite'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

## Convert from pytorch to tflite using Tinynn

In [7]:
!pip install git+https://github.com/alibaba/TinyNeuralNetwork.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/alibaba/TinyNeuralNetwork.git to /tmp/pip-req-build-cbt604dk
  Running command git clone -q https://github.com/alibaba/TinyNeuralNetwork.git /tmp/pip-req-build-cbt604dk
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2 MB 32.4 MB/s 
     |████████████████████████████████| 109 kB 67.1 MB/s 
     |████████████████████████████████| 546 kB 80.7 MB/s 
  Created wheel for TinyNeuralNetwork: filename=TinyNeuralNetwork-0.1.0.20221012234055+186bfdf3b6001ea79788ce564f1bb71bf7a8daa1-py3-none-any.whl size=352174 sha256=4a1d931269fed7a088a0bb56f7c5eca6c313ccf86bec6d9c95fb6f5888335a84
  Stored in directory: /tmp/pip-ephem-wheel-cache-6xkwxsxy/wheels/64/6b/f5/9cf69de054ba0de53e572bc2f13988a664f3dc9623e6f0825d
Successfully built TinyNeuralNetwork


In [8]:
import whisper
import torch

import torch.nn as nn
from tinynn.converter import TFLiteConverter
from tinynn.graph.quantization.quantizer import PostQuantizer
from tinynn.graph.tracer import model_tracer
from tinynn.util.train_util import DLContext, get_device, train

model = whisper.load_model("tiny")
model.eval()
audio = whisper.load_audio("/content/openai-whisper/filename.mp3")
audio = whisper.pad_or_trim(audio)
print(audio.shape)
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)
print(mel.shape)
quantizer = PostQuantizer(model, mel, work_dir='out', config={'asymmetric': True, 'per_tensor': False})
#qat_model = quantizer.quantize()


(480000,)
torch.Size([80, 3000])
